In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import photutils #Aperture and Annulus stuff 
import datetime
from datetime import datetime, timedelta
import juliandate as jd
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)


import astropy
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, CircularAnnulus, ApertureStats, aperture_photometry
from astropy.stats import sigma_clipped_stats
from astropy.table import Column
import glob
import time
#import astroalign as aa

In [2]:

#Sets up df where data from different dates / filters can be stored
RV_Uma_df = pd.DataFrame(columns = ['File ID', 'Filter', 'x-pos', 'y-pos', 'Aperture Sum', 'Median Flux without bkg', 'Sum without bkg', 'Date and Time'])
StandardStar_df = pd.DataFrame(columns = ['File ID', 'Filter', 'x-pos', 'y-pos', 'Aperture Sum', 'Median Flux without bkg', 'Sum without bkg', 'Date and Time'])
RV_Uma_missed = []
StandardStar_missed = []
Filter = ["Blue", "Red", "Green", "Luminance"]

def GetData(filepath, colour):
    
    #resets current image list
    image_list = []
    n = 0
    
    #Gets files in the filepath
    for file in glob.glob(filepath+"*.fits"):
        image_list.append(file)
        n=n+1
    #print("\n".join(image_list)) #Check all files have been found
    
    #Loops 
    for file in image_list:
        #Get the file
        image_file = get_pkg_data_filename(file)
        image_data = fits.getdata(image_file, ext=0)
    
        #Find file data
        fitsFile = fits.open(file)
        date_obs = fitsFile[0].header['DATE-OBS']
        filename = (file.replace(filepath,""))
        time = date_obs[11:19]
        date = date_obs[:10]
        data = fitsFile[0].data
        median = np.median(data)
    
        #setting up starfinder
        tol = 5*median
        fwhm = 10
        FindStars = DAOStarFinder(tol, fwhm)
    
        #finds the stars
        stars = FindStars(data)
    
        #loops through positions of stars and adds values to an array
        positions = []
        
        RV_Uma_added = False
        StandardStar_added = False
        
        for i in range(len(stars)):
            posn = (stars['xcentroid'][i], stars['ycentroid'][i])
            positions.append(posn)
        
            #Aperture and annulus set up
            aperture = CircularAperture((positions[i][0], positions[i][1]), r=3.5*fwhm)
            annulus = CircularAnnulus((positions[i][0], positions[i][1]), r_in=3.5*fwhm, r_out=(3.5*fwhm*np.sqrt(5)))
            aperstats = aperture_photometry(image_data, aperture)
            annulstats =  aperture_photometry(image_data, annulus)

            #Annulus set up and getting data
            annulus_masks = annulus.to_mask(method='center')
            annulus_data = annulus_masks.multiply(image_data)
            annulus_data_1d = annulus_data[annulus_masks.data > 0]
            median_bg = np.median(annulus_data_1d)
        
            #Apature data and set up
            aperture_masks = aperture.to_mask(method='center')
            aperture_data = aperture_masks.multiply(image_data)
            aperture_data_1d = aperture_data[aperture_masks.data > 0]
            
            #Calculations bkg sub etc
            flux = aperstats['aperture_sum'][0] - median_bg * aperture.area
            median_flux_bgsub = np.median(aperture_data_1d) - median_bg
            aperture_stats = ApertureStats(image_data, aperture)
            
            #Takes location of RV Uma and adds the data for it to the data frame 
            if ((1050 < stars['xcentroid'][i] < 1250) & ( 750 < stars['ycentroid'][i] < 1000)):
                RV_Uma_df.loc[len(RV_Uma_df)] = {'File ID': file, 'Filter': colour, 'x-pos': positions[i][0], 'y-pos': positions[i][1], 'Aperture Sum': aperstats['aperture_sum'][0], 'Median Flux without bkg': median_flux_bgsub, 'Sum without bkg':flux, 'Date and Time': date+" "+time}
                RV_Uma_added = True
            
            #Bit below is to selected TYC 3850-738-1
            elif ((250 < stars['xcentroid'][i] < 400) & ( 450 < stars['ycentroid'][i] < 600)):
                StandardStar_df.loc[len(StandardStar_df)] = {'File ID': file, 'Filter': colour, 'x-pos': positions[i][0], 'y-pos': positions[i][1], 'Aperture Sum': aperstats['aperture_sum'][0], 'Median Flux without bkg': median_flux_bgsub, 'Sum without bkg':flux, 'Date and Time': date+" "+time}         
                StandardStar_added = True
        
        #adds file to missed list if the star counldn't be found
        if (RV_Uma_added == False):
            RV_Uma_missed.append(file)
        if(StandardStar_added == False):
            StandardStar_missed.append(file)
       
        print(file) #just to show progress
    
    print("\n",filepath, " has been added",)

In [ ]:
for colour in Filter:
    print(colour)
    GetData("..\\RV Uma\\2022_03_20\\"+colour+"\\", colour)
    GetData("..\\RV Uma\\2022_03_22\\"+colour+"\\", colour)
    GetData("..\\RV Uma\\2022_03_24\\"+colour+"\\", colour)
    GetData("..\\RV Uma\\2023_02_23\\"+colour+"\\", colour)
    GetData("..\\RV Uma\\2023_04_03\\"+colour+"\\", colour)

Blue
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_001.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_002.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_003.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_004.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_005.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_006.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_007.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_008.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_009.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_010.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_011.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_012.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_013.fits
..\RV Uma\2022_03_20\Blue\Reduced_RV_Uma_Light_Blue_32_secs_014.fits
..\RV Uma\2022_03_20\Blue\Red

..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_004.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_005.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_006.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_007.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_008.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_009.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_010.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_011.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_012.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_013.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_014.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_015.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_016.fits
..\RV Uma\2022_03_24\Blue\Reduced_RV_Uma_Light_Blue_32_secs_017.fits
..\RV Uma\2022_03_24\Blue\Reduced_

..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_043.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_044.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_045.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_046.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_047.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_048.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_049.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_050.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_051.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_052.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_053.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_054.fits
..\RV Uma\2022_03_20\Red\Reduced_RV_Uma_Light_Red_32_secs_055.fits

 ..\RV Uma\2022_03_20\Red\  has been added
..\RV Uma\2022_03_22\Red\Reduced_RV_Uma_Light_Red_32_secs_001.fits
..\RV Uma\2022_03_

In [ ]:
print("Where RV_UMa is missing: ", len(RV_Uma_missed))
print("Where Standard Star is missing: ", len(StandardStar_missed))

#removes Standard stars data where RV_Uma is missing and vise versa
for i in range (len(RV_Uma_missed)):
    StandardStar_df.drop(StandardStar_df.index[(StandardStar_df['File ID'] == RV_Uma_missed[i])], axis=0,inplace=True)

for i in range (len(StandardStar_missed)):
    RV_Uma_df.drop(RV_Uma_df.index[(RV_Uma_df['File ID'] == StandardStar_missed[i])], axis=0, inplace=True)
    
#Resets the index so it can loop and match up with eachother correctly
RV_Uma_df = RV_Uma_df.reset_index(drop=True) #The drop stops the old index from being added as a new column
StandardStar_df = StandardStar_df.reset_index(drop=True)

In [ ]:
#Finding the mean of the standard star sum
StandardStarMean = StandardStar_df['Sum without bkg'].mean()
StandardStarRef = StandardStar_df['Sum without bkg'][0]

#finding the differeance between the points and it's average
StandardStar_df['Dif'] = StandardStarMean - StandardStar_df['Sum without bkg']
#StandardStar_df['Dif'] = StandardStarRef - StandardStar_df['Sum without bkg']

#Move all the standard star points so they are on the same line by correction
StandardStar_df['Corrected Sum'] = StandardStar_df['Sum without bkg'] + StandardStar_df['Dif']
RV_Uma_df['Corrected Sum'] = RV_Uma_df['Sum without bkg'] + StandardStar_df['Dif']

#Caculating instrumental mag
RV_Uma_df['Instrumental Magnitude'] = -2.5*np.log10(RV_Uma_df['Corrected Sum'])
StandardStar_df['Instrumental Magnitude'] = -2.5*np.log10(StandardStar_df['Corrected Sum'])

In [ ]:
StandardStar_df
StandardStar_mean = np.mean(StandardStar_df['Corrected Sum'])
StandardStar_mean

In [ ]:
RV_Uma_df
RV_Uma_mean = np.mean(RV_Uma_df['Corrected Sum'])
RV_Uma_mean

In [ ]:
Average_instrumental = -2.5*np.log10(StandardStar_mean)
StandardStar_df['Average_instrumental'] = Average_instrumental - StandardStar_df['Instrumental Magnitude']
StandardStar_df['Correct instrumental'] =  StandardStar_df['Average_instrumental'] + StandardStar_df['Instrumental Magnitude']
#StandardStar_df

In [ ]:
RV_Uma_df['Correct Instrumental'] = StandardStar_df['Average_instrumental'] + RV_Uma_df['Instrumental Magnitude']
#RV_Uma_df

In [ ]:
def Phase(DataFrame, Period):

    #Only work if the column is added first for some reason
    DataFrame['Phase'] = 0
    
    for i in range(len(DataFrame)):
    #Picks out corresponding values
        year = int((DataFrame['Date and Time'][i])[0:4])
        month = int((DataFrame['Date and Time'][i])[5:7])
        day = int((DataFrame['Date and Time'][i])[8:10])
        hour = int((DataFrame['Date and Time'][i])[11:13])
        minute = int((DataFrame['Date and Time'][i])[14:16])
        second = int((DataFrame['Date and Time'][i])[17:19])

        #Caculates Julian Date and phase
        J_D = jd.from_gregorian(year, month, day, hour, minute, second, 0)
        P = ((J_D/Period) - np.floor(J_D/Period))

        #Adds the phase 
        DataFrame['Phase'][i] = P

In [ ]:
Period = 0.468060 #of RV_Uma

Phase(RV_Uma_df, Period)
Phase(StandardStar_df, Period)

In [ ]:
#Plots counts agasint time of
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1)


"""
#In flux
ax.plot(RV_Uma_df['Phase'], RV_Uma_df['Corrected Sum'].where(RV_Uma_df['Filter'] == colour), marker = 'o', markersize=2, color = 'blue', linestyle='none', label="RV Uma") 
ax.plot(RV_Uma_df['Phase']+1, RV_Uma_df['Corrected Sum'].where(RV_Uma_df['Filter'] == colour), marker = 'o', markersize=2, color = 'blue', linestyle='none') 

ax.plot(StandardStar_df['Phase'], StandardStar_df['Corrected Sum'], marker = 'o', markersize=2, color = 'black', linestyle='none', label="Standard Star") 
ax.plot(StandardStar_df['Phase']+1, StandardStar_df['Corrected Sum'], marker = 'o', markersize=2, color = 'black', linestyle='none') 
ax.set_ylabel('Flux')
"""


#In Instrumental Magnitude
ax.plot(RV_Uma_df['Phase'], RV_Uma_df['Apparent Magnitude'].where(RV_Uma_df['Filter'] == "Blue"), marker = 'o', markersize=2, color = 'blue', linestyle='none', label="RV Uma") 
ax.plot(RV_Uma_df['Phase']+1, RV_Uma_df['Apparent Magnitude'].where(RV_Uma_df['Filter'] == "Blue"), marker = 'o', markersize=2, color = 'blue', linestyle='none') 

ax.plot(RV_Uma_df['Phase'], RV_Uma_df['Apparent Magnitude'].where(RV_Uma_df['Filter'] == "Red"), marker = 'o', markersize=2, color = 'red', linestyle='none', label="RV Uma") 
ax.plot(RV_Uma_df['Phase']+1, RV_Uma_df['Apparent Magnitude'].where(RV_Uma_df['Filter'] == "Red"), marker = 'o', markersize=2, color = 'red', linestyle='none') 

ax.plot(RV_Uma_df['Phase'], RV_Uma_df['Apparent Magnitude'].where(RV_Uma_df['Filter'] == "Green"), marker = 'o', markersize=2, color = 'green', linestyle='none', label="RV Uma") 
ax.plot(RV_Uma_df['Phase']+1, RV_Uma_df['Apparent Magnitude'].where(RV_Uma_df['Filter'] == "Green"), marker = 'o', markersize=2, color = 'green', linestyle='none')

ax.plot(RV_Uma_df['Phase'], RV_Uma_df['Apparent Magnitude'].where(RV_Uma_df['Filter'] == "Luminance"), marker = 'o', markersize=2, color = 'black', linestyle='none', label="RV Uma") 
ax.plot(RV_Uma_df['Phase']+1, RV_Uma_df['Apparent Magnitude'].where(RV_Uma_df['Filter'] == "Luminance"), marker = 'o', markersize=2, color = 'black', linestyle='none') 
#ax.plot(StandardStar_df['Phase'], StandardStar_df['Instrumental Magnitude'], marker = 'o', markersize=2, color = 'black', linestyle='none', label="TYC 3850-738-1") 
#ax.plot(StandardStar_df['Phase']+1, StandardStar_df['Instrumental Magnitude'], marker = 'o', markersize=2, color = 'black', linestyle='none') 

ax.invert_yaxis()
ax.set_ylabel('Apparent Magnitude')

ax.set_xlabel('Phase')

ax.legend()